# XOR Problem

In [4]:
import numpy as np
import sklearn.neural_network
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

criterio_convergencia = 0.3

x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) # [0, 0, 0, 1, 1, 0, 1, 1]).reshape(4, 2)
# y = np.array([  0,      1,      1,   0]) # .reshape(4,)
y = np.array([-1, 1, 1, -1])
print(x)
print(y)

[[0 0]
 [0 1]
 [1 0]
 [1 1]]
[-1  1  1 -1]


In [7]:
LRS = [0.05, 0.25, 0.5, 1, 2]
momentos = [0, 0.5, 1]
shuffles = [True, False]
matrixs = []
matrix = []
indexs = []

for mom in momentos:
    matrix = []
    for lr in LRS:
        indexs = []
        for n in range(10):
            model = MLPClassifier(activation='tanh',  #relu logistic tanh identity
                                 solver='sgd',
                                 max_iter=30000, 
                                 hidden_layer_sizes=(3,),
                                 learning_rate_init=lr,
                                 shuffle=False,
                                 momentum=mom,
                                 tol=0.000001,
                                 n_iter_no_change=500).fit(x, y)

            boolean = True
            for index in range(len(model.loss_curve_)):
                if model.loss_curve_[index] <= criterio_convergencia:
                    indexs.append(index+1)
                    boolean = False
                    break
            if boolean:
                indexs.append("nan")            
        matrix.append(indexs)
    #print("Momento:", mom,"matrix:\n", np.array(matrix).transpose())
    matrixs.append(np.array(matrix).transpose())

In [8]:
print("momento 0")
pd.DataFrame(matrixs[0])

momento 0


,0,1,2,3,4
0,386,132,91,74,69
1,987,367,76,90,21
2,nan,96,nan,65,18
3,863,133,92,nan,22
4,1643,216,96,44,27
5,771,247,76,47,52
6,1453,424,184,44,22
7,1100,193,97,99,nan
8,nan,151,62,89,17
9,1139,315,89,64,16


# Efeito da introdução de um termo de momento

In [9]:
print("momento 0.5")
pd.DataFrame(matrixs[1])

momento 0.5


,0,1,2,3,4
0,234,83,25,18,17
1,696,118,34,54,19
2,662,106,45,20,19
3,2853,70,25,32,13
4,243,181,51,23,19
5,494,72,78,nan,15
6,723,148,77,22,12
7,732,98,61,24,25
8,nan,79,149,27,25
9,313,296,35,27,12


In [10]:
print("momento 1")
pd.DataFrame(matrixs[2])

momento 1


,0,1,2,3,4
0,55,15,nan,13,10
1,93,37,nan,20,13
2,nan,41,11,14,nan
3,94,nan,28,19,nan
4,nan,52,19,18,17
5,69,nan,nan,14,nan
6,54,32,12,8,14
7,72,56,25,nan,nan
8,103,nan,22,10,10
9,36,52,16,14,8


## Perceber os melhores parametros

In [11]:
x, y

(array([[0, 0],
        [0, 1],
        [1, 0],
        [1, 1]]), array([-1,  1,  1, -1]))

In [12]:
LRS = [0.05, 0.25, 0.5, 1, 2]
momentos = [0, 0.5, 1]
shuffles = [True, False]

mlp = MLPClassifier(activation =  "tanh",
                    solver = 'sgd', 
                    max_iter = 30000,
                    tol = 0.000001,
                    n_iter_no_change = 500).fit(x, y)

mlp_params = {
    "hidden_layer_sizes" : [(2, ), (4,), (4, 2), (3,)],
    'learning_rate_init': [0.05, 0.25, 0.5, 1, 2], # np.arange(0.1, 1, 0.2), # [0.1, 0.8, 1], "activation" : ["logistic","relu", "tanh"],
    "momentum" : [0, 0.5, 1],
    "shuffle": [True, False]
}
mlp_cv = GridSearchCV(mlp, mlp_params, cv = 2, verbose = 1).fit(x, y)

#optimum param
print(mlp_cv.best_params_)
print(mlp_cv.best_score_)

Fitting 2 folds for each of 120 candidates, totalling 240 fits
{'hidden_layer_sizes': (2,), 'learning_rate_init': 0.5, 'momentum': 1, 'shuffle': False}
0.5


In [18]:
model = MLPClassifier(activation='tanh',  #relu logistic tanh identity
                     solver='sgd',
                     max_iter=30000, 
                     hidden_layer_sizes=(2,),
                     learning_rate_init=0.5,
                     shuffle=True,
                     momentum=1,
                     tol=0.000001,
                     n_iter_no_change=500).fit(x, y)

print("Prever o padrao A:", model.predict([x[0]]), np.array_equal(model.predict([x[0]]), np.array([-1])))
print("Prever o padrao B:", model.predict([x[1]]), np.array_equal(model.predict([x[1]]), np.array([ 1])))
print("Prever o padrao C:", model.predict([x[2]]), np.array_equal(model.predict([x[2]]), np.array([ 1,])))
print("Prever o padrao D:", model.predict([x[3]]), np.array_equal(model.predict([x[3]]), np.array([-1])))

# Score
print("Score:", model.score(x, y)*100)

Prever o padrao A: [1] False
Prever o padrao B: [1] True
Prever o padrao C: [1] True
Prever o padrao D: [-1] True
Score: 75.0


# Ordem de apresentação aleatória

In [71]:
LRS = [0.05, 0.25, 0.5, 1, 2]
matrix = []
indexs = []

for lr in LRS:
    indexs = []
    for n in range(10):
        model = MLPClassifier(activation='tanh',  #relu logistic tanh identity
                             solver='sgd',
                             max_iter=30000, 
                             hidden_layer_sizes=(2,),
                             learning_rate_init=lr,
                             shuffle=True,
                             momentum=1,
                             tol=0.000001,
                             n_iter_no_change=500).fit(x, y)

        boolean = True
        for index in range(len(model.loss_curve_)):
            if model.loss_curve_[index] <= criterio_convergencia:
                indexs.append(index+1)
                boolean = False
                break
        if boolean:
            indexs.append("nan")            
    matrix.append(indexs)

In [72]:
print("Ordem de apresentação aleatória")
pd.DataFrame(np.array(matrix).transpose())

Ordem de apresentação aleatória


,0,1,2,3,4
0,80,nan,nan,nan,nan
1,nan,66,nan,13,nan
2,nan,nan,nan,17,nan
3,nan,30,nan,115,nan
4,51,30,23,nan,nan
5,88,50,nan,nan,nan
6,nan,nan,43,15,14
7,89,59,nan,nan,nan
8,nan,40,nan,606,18
9,nan,nan,36,nan,nan


# Efeito de utilização de uma codificação binária ou bipolar

In [35]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]) 
#y = np.array([  0,      1,      1,      0])
y = np.array([[-1, 1], [1, -1], [1, -1], [-1, 1]])


In [36]:
LRS = [0.05, 0.25, 0.5, 1, 2]
matrix = []
indexs = []

for lr in LRS:
    indexs = []
    for n in range(10):
        model = MLPClassifier(activation='tanh',  #relu logistic tanh identity
                             solver='sgd',
                             max_iter=30000, 
                             hidden_layer_sizes=(2, ),
                             learning_rate_init=lr,
                             shuffle=True,
                             momentum=0.5,
                             tol=0.000001,
                             n_iter_no_change=500).fit(x, y)

        boolean = True
        for index in range(len(model.loss_curve_)):
            if model.loss_curve_[index] <= criterio_convergencia:
                indexs.append(index+1)
                boolean = False
                break
        if boolean:
            indexs.append("nan")            
    matrix.append(indexs)

In [37]:
print("Efeito de utilização de uma codificação binária ou bipolar;")
pd.DataFrame(np.array(matrix).transpose())

Efeito de utilização de uma codificação binária ou bipolar;


,0,1,2,3,4
0,26,6,4,2,2
1,31,9,4,4,2
2,49,8,7,3,2
3,32,8,4,3,2
4,36,7,4,3,2
5,36,8,4,3,2
6,35,10,4,3,2
7,19,6,3,3,2
8,34,7,4,3,2
9,27,6,5,3,2
